In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [31]:
# 단계 1 : 문서 로드
loader = CSVLoader(r"C:\LangGraph\Diseases_Symptoms.csv")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")
print(docs[0].metadata)

문서의 페이지수: 400
{'source': 'C:\\LangGraph\\Diseases_Symptoms.csv', 'row': 0}


In [32]:
# CSV 파일 경로
loader = CSVLoader(
    file_path="./Diseases_Symptoms.csv",
    csv_args={
        "delimiter": ",",  # 구분자
        "quotechar": '"',  # 인용 부호 문자
        "fieldnames": [
            "Code",
            "Name",
            "Symptoms",
            "Treatments",
        ],  # 필드 이름
    },
)

# 데이터 로드
docs = loader.load()

# 데이터 출력
print(docs[1].page_content)


Code: 1
Name: Panic disorder
Symptoms: Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness
Treatments: Antidepressant medications, Cognitive Behavioral Therapy, Relaxation Techniques


In [33]:
# 단계 2 : 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 402


In [34]:
# 단계 3 : 임베딩 생성
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs={"device": "cuda"}, encode_kwargs={"normalize_embeddings": True})

In [35]:
# 단계 4 : DB 생성 및 저장
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings)

In [36]:
# 단계 5: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

In [37]:
# 관련 문서를 검색
docs = retriever.invoke("나 콧물 나와")

for doc in docs:
    print(doc.page_content)
    print("=========================================================")


Code: 322
Name: Peritonsillar Abscess
Symptoms: Severe sore throat, difficulty swallowing, fever, swollen tonsils, voice changes, neck pain or stiffness
Treatments: Antibiotics to treat infection, drainage of the abscess (needle aspiration or incision and drainage), pain relievers, warm saltwater gargles, rest, fluids, surgical removal of the tonsils (in chronic or recurrent cases)
Code: 322
Name: Peritonsillar Abscess
Symptoms: Severe sore throat, difficulty swallowing, fever, swollen tonsils, voice changes, neck pain or stiffness
Treatments: Antibiotics to treat infection, drainage of the abscess (needle aspiration or incision and drainage), pain relievers, warm saltwater gargles, rest, fluids, surgical removal of the tonsils (in chronic or recurrent cases)
Code: 322
Name: Peritonsillar Abscess
Symptoms: Severe sore throat, difficulty swallowing, fever, swollen tonsils, voice changes, neck pain or stiffness
Treatments: Antibiotics to treat infection, drainage of the abscess (needle a

In [38]:
# 단계 7: 언어 모델 생성
llm=ChatGroq(temperature=0, model="gemma2-9b-it", api_key="")

In [39]:
llm.invoke("나 콧물 나와")

AIMessage(content='콧물이 나오는 건 몸이 감기에 걸렸거나 알레르기 반응을 보이는 신호일 수 있어요. \n\n무엇이 콧물을 유발하는지 알아보는 게 중요해요. \n\n* **감기 증상인가요?** 콧물 외에도 기침, 목의 가려움, 열, 두통 등 다른 증상이 있나요?\n* **알레르기 반응인가요?** 꽃가루, 먼지, 특정 음식 등 특정 물질에 노출되었나요?\n* **기타 원인은 없는지 생각해 보세요.** \n\n콧물이 심하거나 다른 증상이 동반된다면, 병원에 방문하여 진료를 받는 것이 좋습니다. \n\n건강을 챙기세요!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 192, 'prompt_tokens': 15, 'total_tokens': 207, 'completion_time': 0.349090909, 'prompt_time': 0.000220119, 'queue_time': 0.096715231, 'total_time': 0.349311028}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d39f3ce5-01bf-4a5b-9d22-ed31a751763f-0', usage_metadata={'input_tokens': 15, 'output_tokens': 192, 'total_tokens': 207})

In [40]:
prompt = PromptTemplate.from_template(
    """You are an assistant in the question-and-answer task. However, this content is for medical information only. Use the following retrieved context fragment to answer the question. 
If you don't know the answer, just say you don't know. Please answer in Korean. Displays up to three related diseases along with their symptoms, treatments, and match rates. The matching rate is extracted as a percentage of the searched value.

#question: 
{question} 
#context: 
{context} 

#answer:
1. 질병명: 
   매칭률:
   증상: 
   치료법: 

2. 질병명: 
   매칭률: 
   증상: 
   치료법: 

3. 질병명: 
   매칭률: 
   증상: 
   치료법:
"""
)

In [41]:
# 단계 8: 체인(chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [42]:
question = "재채기가 나오고 눈이 가려워"

response = chain.invoke(question)

print(response)


1. 질병명: 알레르기성 결막염
   매칭률: 높음 
   증상: 충혈, 가려움, 맑은 분비물, 눈꺼풀 부종
   치료법: 알레르겐 피하기, 항히스타민 안약, 차가운 찜질, 윤활 안약

2. 질병명: 안내막염
   매칭률: 낮음
   증상: 눈 통증, 충혈, 붓기, 시력 저하, 시력 감소, 흐릿한 물체(시야 내 어두운 반점 또는 거미줄 모양) , 빛에 대한 민감도 증가, 눈물 분비물, 과도한 눈물 
   치료법: 즉시 의료적 치료가 필요합니다. 정맥 내 항생제 또는 항진균제, 안내막 주사(항생제 또는 항진균제를 직접 눈 안에 주사), 안내막절제술(눈 안의 젤리 같은 액체를 제거하는 수술), 국소 또는 경구 약물(통증, 염증 또는 감염을 관리하기 위해), 안과 전문의와의 자주 측정, 보조적 치료(따뜻한 찜질 또는 윤활 안약과 같이)

3. 질병명:  
   매칭률: 
   증상:  
   치료법: 




